#### The following notebooks shows the integrations between DuckDB spatial and loanboard library to display data from Overture Map building.

In [2]:
import duckdb
import lonboard

In [3]:
# Connect to DuckDB and install extensions
db = duckdb.connect()
db.execute("""
    INSTALL spatial;
    LOAD spatial;

    INSTALL httpfs;
    LOAD httpfs;
    SET s3_region='us-west-2';
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
# Define Buenos Aires bounding box coordinates
buenos_aires_bbox = """
SELECT ST_GeomFromText('POLYGON((
    -58.3700 -34.6200,
    -58.3700 -34.5900,
    -58.3550 -34.5900,
    -58.3550 -34.6200,
    -58.3700 -34.6200
))') as geometry
"""

# Query buildings from Overture Maps
buildings_query = f"""
SELECT
  id,
  geometry,
  height,
  class as building_type
FROM read_parquet('s3://overturemaps-us-west-2/release/2025-03-19.0/theme=buildings/type=building/*.parquet', hive_partitioning=1)
WHERE bbox.xmin BETWEEN -58.53 AND -58.34
  AND bbox.ymin BETWEEN -34.71 AND -34.52
  AND height > 0
"""

# Execute query and create table
db.execute("CREATE OR REPLACE TABLE buenos_aires_buildings AS " + buildings_query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
# Create map visualization
layer = lonboard.PolygonLayer.from_duckdb(
    sql="SELECT * FROM buenos_aires_buildings",
    con=db,
    get_fill_color=[200, 100, 0, 180],
    get_line_color=[0, 0, 0, 255],
    get_line_width=1,
    pickable=True,
    extruded=True,
    get_elevation=db.table('buenos_aires_buildings').arrow()["height"],
    auto_highlight=True
)

c:\Users\learo\MyDocs\GitHub\geopython\.venv\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [7]:
map = lonboard.Map(
    layers=[layer],
    view_state={
        "longitude": -58.45153,
        "latitude": -34.58158,
        "zoom": 13,
        "pitch": 45,
        "bearing": 0
    }
)

map

Map(custom_attribution='', layers=(PolygonLayer(auto_highlight=True, extruded=True, get_elevation=arro3.core.C…